In [1]:
#Standard python library imports
import pandas as pd
import geopandas as gp
import os
from collections import Counter
#Py file fxn import
from helper_functions import *
pd.options.display.max_columns = 100


#RDH created Precinct Boundary and Election Results file
pber = gp.read_file("../nc_2022_gen_prec_no_splits/nc_2022_gen_prec.shp")
#2022 NC congressional plan
cong = gp.read_file("./nc_cong_adopted_2022/NC_SMmap2_Statewide.shp")
cong = cong.to_crs(pber.crs)


#Election gdf to be modified
pber["CONG_DIST"] = 0
pber_cong = pber[["UNIQUE_ID","COUNTYFP","COUNTYNM","PRECINCT","CONG_DIST"]+list(pber.columns[(pber.columns.str.contains("GCON"))])+["geometry"]]

# STATEWIDE - US SENATE + COURT POSITIONS

In [2]:
pber['GSAC08DTHO'] = pber["GIA08DTHO"]
pber['GSAC08RFLO'] = pber["GIA08RFLO"]
pber['GSAC09DSAL'] = pber["GIA09DSAL"]
pber['GSAC09RSTR'] = pber["GIA09RSTR"]
pber['GSAC10DADA'] = pber["GIA10DADA"]
pber['GSAC10RTYS'] = pber["GIA10RTYS"]
pber['GSAC11DJAC'] = pber["GIA11DJAC"]
pber['GSAC11RSTA'] = pber["GIA11RSTA"]


#Statewide gdf - all set to export as shp
statewide = pber[["UNIQUE_ID","COUNTYFP","COUNTYNM","PRECINCT"]+list(pber.columns[(pber.columns.str.contains("USS"))|(pber.columns.str.contains("SSC"))|(pber.columns.str.contains("SAC"))])+["geometry"]]

## Prepare for Export

In [3]:
statewide.to_file("./nc_2022_gen_prec/nc_gen_22_st_prec.shp")

# US HOUSE

In [4]:
def is_split_precinct(district_assignment_list):
    '''
    input: list of precincts and district assignments
    fxn: evaluate if precinct has one or more district assignments
    return: 0 if precinct:district is 1:1 assignment, or return assignment 
    '''
    c = Counter([x[0] for x in district_assignment_list])
    greater_than_one = {x:[y[1] for y in district_assignment_list if y[0]==x] for x, count in c.items() if count > 1}
    if len(greater_than_one)==0:
        return 0
    else:
        return greater_than_one
    
def get_level_dist(column_name):
    '''
    fxn to grab district and contest from column names
    '''
    if column_name[0:4] == "GSLP":
        level = "SL"
        dist = "004"
    elif column_name[0:3] == "GSL":
        level = "SL"
        dist = column_name[3:6]
    elif column_name[0:3] == "GCO":
        level = "CON"
        dist = column_name[4:6]
    else:
        print(column_name)
        raise ValueError
    return level,dist


def return_cd(row):
    for val in races:
        if row[val] != 0:
            return val[4:6]
        

races = [i for i in list(pber_cong.columns) if "GCON" in i]
precinct_mapping_dict = {}
split_precincts_list = {}
for index,row in pber_cong.iterrows():
    precinct_list = []
    for contest in races:
        if(row[contest]!=0):
            precinct_info = get_level_dist(contest)
            if precinct_info not in precinct_list:
                precinct_list.append(get_level_dist(contest))
    is_split = is_split_precinct(precinct_list)
    if (is_split):
        split_precincts_list[row["UNIQUE_ID"]]=is_split
    precinct_mapping_dict[row["UNIQUE_ID"]]=precinct_list
    
      
#Standardize district format with name different from PBER to use later and distinguish when splits occur
cong['HOUSE_DIST']=cong["District_A"].astype(str).str.zfill(2)
pber_cong["CONG_DIST"] = pber_cong.apply(lambda row: return_cd(row), axis = 1)


split_precincts_list

/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


{'CABARRUS---01-08': {'CON': ['08', '12']},
 'FORSYTH---021': {'CON': ['05', '06']},
 'MECKLENBURG---PCT 027': {'CON': ['12', '14']},
 'RICHMOND---MARKS CREEK 2': {'CON': ['08', '09']},
 'WAKE---PRECINCT 10-05': {'CON': ['02', '13']},
 'PITT---GREENVILLE 13B': {'CON': ['01', '03']},
 'RUTHERFORD---28': {'CON': ['10', '11']},
 'GASTON---38': {'CON': ['10', '14']},
 'CUMBERLAND---CC519': {'CON': ['07', '09']},
 'CASWELL---MILTON': {'CON': ['04', '06']},
 'CALDWELL---DUDLEY SHOALS': {'CON': ['05', '10']},
 'HARNETT---PR20': {'CON': ['09', '13']}}

In [5]:
#Initialize new variable name to update in for loop
pber_w_splits = pber_cong.copy()
for val in pber_cong["UNIQUE_ID"]:
    cd_list = []
    if val in split_precincts_list.keys():
        print(val)
        pber_w_splits = district_splits_mod(split_precincts_list[val],"CONG",val, pber_w_splits, cong, "UNIQUE_ID", "HOUSE_DIST", races, "CONG_DIST")

CABARRUS---01-08
FORSYTH---021
MECKLENBURG---PCT 027
RICHMOND---MARKS CREEK 2
WAKE---PRECINCT 10-05
PITT---GREENVILLE 13B
RUTHERFORD---28
GASTON---38
CUMBERLAND---CC519
CASWELL---MILTON
CALDWELL---DUDLEY SHOALS
HARNETT---PR20


In [6]:
pber_w_splits[pber_w_splits['UNIQUE_ID'].str.contains("CONG")]

,UNIQUE_ID,COUNTYFP,COUNTYNM,PRECINCT,CONG_DIST,GCON01DDAV,GCON01RSMI,GCON02DROS,GCON02RVIL,GCON03DGAS,GCON03RMUR,GCON04DFOU,GCON04RGEE,GCON05DPAR,GCON05RFOX,GCON06DMAN,GCON06LWAT,GCON06RCAS,GCON07DGRA,GCON07RROU,GCON08DHUF,GCON08RBIS,GCON09DCLA,GCON09RHUD,GCON10DGEN,GCON10OWRI,GCON10RMCH,GCON11DBEA,GCON11LCOA,GCON11REDW,GCON12DADA,GCON12RLEE,GCON13DNIC,GCON13RHIN,GCON14DJAC,GCON14RHAR,geometry
2643,CABARRUS---01-08-(CONG-08),25,CABARRUS,01-08,08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,306,156,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"MULTIPOLYGON (((1524207.930 569177.259, 152428..."
2644,CABARRUS---01-08-(CONG-12),25,CABARRUS,01-08,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1139,978,0,0,0,0,"POLYGON ((1530841.816 572533.017, 1530892.011 ..."
2645,FORSYTH---021-(CONG-05),67,FORSYTH,021,05,0,0,0,0,0,0,0,0,468,1710,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((1664254.662 894718.024, 1664600.457 ..."
2646,FORSYTH---021-(CONG-06),67,FORSYTH,021,06,0,0,0,0,0,0,0,0,0,0,1,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"MULTIPOLYGON (((1674559.455 885389.531, 167456..."
2647,MECKLENBURG---PCT 027-(CONG-12),119,MECKLENBURG,PCT 027,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,699,80,0,0,0,0,"POLYGON ((1452888.603 548755.901, 1453110.149 ..."
2648,MECKLENBURG---PCT 027-(CONG-14),119,MECKLENBURG,PCT 027,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,39,5,"POLYGON ((1453240.594 545837.708, 1453076.369 ..."
2649,RICHMOND---MARKS CREEK 2-(CONG-08),153,RICHMOND,MARKS CREEK 2,08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,624,984,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((1812231.115 401251.284, 1812363.737 ..."
2650,RICHMOND---MARKS CREEK 2-(CONG-09),153,RICHMOND,MARKS CREEK 2,09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,227,458,0,0,0,0,0,0,0,0,0,0,0,0,"MULTIPOLYGON (((1802023.861 388338.237, 180162..."
2651,WAKE---PRECINCT 10-05-(CONG-02),183,WAKE,PRECINCT 10-05,02,0,0,1122,1029,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((2155108.593 721428.714, 2155112.019 ..."
2652,WAKE---PRECINCT 10-05-(CONG-13),183,WAKE,PRECINCT 10-05,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,168,137,0,0,"POLYGON ((2161259.128 712830.393, 2161140.803 ..."


In [7]:
pber_w_splits[pber_w_splits["CONG_DIST"].isna()]

,UNIQUE_ID,COUNTYFP,COUNTYNM,PRECINCT,CONG_DIST,GCON01DDAV,GCON01RSMI,GCON02DROS,GCON02RVIL,GCON03DGAS,GCON03RMUR,GCON04DFOU,GCON04RGEE,GCON05DPAR,GCON05RFOX,GCON06DMAN,GCON06LWAT,GCON06RCAS,GCON07DGRA,GCON07RROU,GCON08DHUF,GCON08RBIS,GCON09DCLA,GCON09RHUD,GCON10DGEN,GCON10OWRI,GCON10RMCH,GCON11DBEA,GCON11LCOA,GCON11REDW,GCON12DADA,GCON12RLEE,GCON13DNIC,GCON13RHIN,GCON14DJAC,GCON14RHAR,geometry


In [8]:
pber_w_splits.shape

(2668, 37)

### Check vote totals post-split

In [9]:
county_totals_check(pber, "pber", pber_w_splits, "join",races, "COUNTYFP")

***Countywide Totals Check***

GCON01DDAV is equal across all counties
GCON01RSMI is equal across all counties
GCON02DROS is equal across all counties
GCON02RVIL is equal across all counties
GCON03DGAS is equal across all counties
GCON03RMUR is equal across all counties
GCON04DFOU is equal across all counties
GCON04RGEE is equal across all counties
GCON05DPAR is equal across all counties
GCON05RFOX is equal across all counties
GCON06DMAN is equal across all counties
GCON06LWAT is equal across all counties
GCON06RCAS is equal across all counties
GCON07DGRA is equal across all counties
GCON07RROU is equal across all counties
GCON08DHUF is equal across all counties
GCON08RBIS is equal across all counties
GCON09DCLA is equal across all counties
GCON09RHUD is equal across all counties
GCON10DGEN is equal across all counties
GCON10OWRI is equal across all counties
GCON10RMCH is equal across all counties
GCON11DBEA is equal across all counties
GCON11LCOA is equal across all counties
GCON11RED

In [10]:
statewide_totals_check(pber, "pber", pber_w_splits, "join",races)

***Statewide Totals Check***
GCON01DDAV is equal 	 both dataframes 134802
GCON01RSMI is equal 	 both dataframes 122755
GCON02DROS is equal 	 both dataframes 190722
GCON02RVIL is equal 	 both dataframes 104168
GCON03DGAS is equal 	 both dataframes 82407
GCON03RMUR is equal 	 both dataframes 166500
GCON04DFOU is equal 	 both dataframes 194866
GCON04RGEE is equal 	 both dataframes 96500
GCON05DPAR is equal 	 both dataframes 102329
GCON05RFOX is equal 	 both dataframes 175245
GCON06DMAN is equal 	 both dataframes 139586
GCON06LWAT is equal 	 both dataframes 2810
GCON06RCAS is equal 	 both dataframes 116735
GCON07DGRA is equal 	 both dataframes 120220
GCON07RROU is equal 	 both dataframes 164048
GCON08DHUF is equal 	 both dataframes 79248
GCON08RBIS is equal 	 both dataframes 184007
GCON09DCLA is equal 	 both dataframes 101227
GCON09RHUD is equal 	 both dataframes 131461
GCON10DGEN is equal 	 both dataframes 73174
GCON10OWRI is equal 	 both dataframes 352
GCON10RMCH is equal 	 both datafram

### Check district assignments

In [11]:
## Intersect the District with GA_CONG to fill in the n/a values
na_values = pber_w_splits[pber_w_splits["CONG_DIST"].isna()]
others = pber_w_splits[~(pber_w_splits["CONG_DIST"].isna())]


print(pber_w_splits.shape)
print(others.shape)
print(na_values.shape)

(2668, 37)
(2668, 37)
(0, 37)


In [12]:
intersected = gp.overlay(cong, na_values, how = "intersection")


# Create a dictionary to map
intersected['area'] = intersected.area
na_assignment_dict = {}
for val in intersected["UNIQUE_ID"].unique():

    assignment = intersected.loc[intersected["UNIQUE_ID"] == val].nlargest(1, 'area')["HOUSE_DIST"].values[0]
    na_assignment_dict[val] = assignment
    
    
pber_w_splits["CONG_DIST"] = pber_w_splits["UNIQUE_ID"].map(na_assignment_dict).fillna(pber_w_splits["CONG_DIST"])


assigned_districts = pber_w_splits.dissolve(["CONG_DIST"])
assigned_districts.reset_index(drop=False,inplace=True)


cong["CONG_DIST"] = cong["HOUSE_DIST"]


compare_geometries(assigned_districts, cong ,"Election Results", "Official Map", "CONG_DIST",area_threshold=.1)

Checking 14 precincts for differences of greater than 0.1 km^2


Scroll down to see plots of any differences

Of the 14 precincts:

0 precincts w/ a difference of 0 km^2
14 precincts w/ a difference between 0 and .1 km^2
0 precincts w/ a difference between .1 and .5 km^2
0 precincts w/ a difference between .5 and 1 km^2
0 precincts w/ a difference between 1 and 2 km^2
0 precincts w/ a difference between 2 and 5 km^2
0 precincts w/ a difference greater than 5 km^2


## Prepare to export

In [13]:
pber_w_splits

,UNIQUE_ID,COUNTYFP,COUNTYNM,PRECINCT,CONG_DIST,GCON01DDAV,GCON01RSMI,GCON02DROS,GCON02RVIL,GCON03DGAS,GCON03RMUR,GCON04DFOU,GCON04RGEE,GCON05DPAR,GCON05RFOX,GCON06DMAN,GCON06LWAT,GCON06RCAS,GCON07DGRA,GCON07RROU,GCON08DHUF,GCON08RBIS,GCON09DCLA,GCON09RHUD,GCON10DGEN,GCON10OWRI,GCON10RMCH,GCON11DBEA,GCON11LCOA,GCON11REDW,GCON12DADA,GCON12RLEE,GCON13DNIC,GCON13RHIN,GCON14DJAC,GCON14RHAR,geometry
0,BURKE---0001,23,BURKE,0001,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,439,2,1108,0,0,0,0,0,0,0,0,0,"POLYGON ((1233595.464 737538.312, 1233589.172 ..."
1,STANLY---0003,167,STANLY,0003,08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,494,680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((1644857.853 584760.831, 1644768.728 ..."
2,BURKE---0003,23,BURKE,0003,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,134,1,498,0,0,0,0,0,0,0,0,0,"POLYGON ((1220715.101 726879.358, 1220723.026 ..."
3,STANLY---0007,167,STANLY,0007,08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,381,739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((1650578.509 584607.573, 1650484.945 ..."
4,STANLY---0008,167,STANLY,0008,08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,479,89,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((1646657.855 579294.538, 1646677.799 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2663,CALDWELL---DUDLEY SHOALS-(CONG-05),27,CALDWELL,DUDLEY SHOALS,05,0,0,0,0,0,0,0,0,118,729,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"MULTIPOLYGON (((1275781.780 762505.816, 127561..."
2664,CALDWELL---DUDLEY SHOALS-(CONG-10),27,CALDWELL,DUDLEY SHOALS,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,65,0,479,0,0,0,0,0,0,0,0,0,"POLYGON ((1290629.287 759203.938, 1290595.493 ..."
2665,HARNETT---PR20-(CONG-07),85,HARNETT,PR20,07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"MULTIPOLYGON (((2045534.669 546970.906, 204545..."
2666,HARNETT---PR20-(CONG-09),85,HARNETT,PR20,09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,71,77,0,0,0,0,0,0,0,0,0,0,0,0,"MULTIPOLYGON (((2039555.027 545998.955, 203980..."


In [14]:
pber_w_splits.to_file("./nc_2022_gen_prec/nc_gen_22_cong_prec.shp")